## onnx -> TFLITE 코드 

### on2tflite

In [1]:
import onnx

model = onnx.load("Last_P2_Lens.onnx")
# /Users/parkjunhui/Desktop/Spresto/480_retinanet_Lens.onnx
onnx.checker.check_model(model)
print("Model is valid!")

Model is valid!


In [2]:
# 이제 시스템 명령어 실행
!onnx2tf \
  -i Last_P2_Lens.onnx\
  -o Last_P2_Lens_tf \
  --copy_onnx_input_output_names_to_tflite
  



Model optimizing started ============================================================
Simplifying...
Finish! Here is the difference:
┏━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━┓
┃            ┃ Original Model ┃ Simplified Model ┃
┡━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━┩
│ Add        │ 3              │ 3                │
│ Constant   │ 46             │ 46               │
│ Conv       │ 23             │ 23               │
│ MaxPool    │ 1              │ 1                │
│ Relu       │ 18             │ 18               │
│ Model Size │ 21.8MiB        │ 21.8MiB          │
└────────────┴────────────────┴──────────────────┘

Simplifying...
Finish! Here is the difference:
┏━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━┓
┃            ┃ Original Model ┃ Simplified Model ┃
┡━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━┩
│ Add        │ 3              │ 3                │
│ Constant   │ 46             │ 46               │
│ Conv       │ 23             │ 23               │
│ 

### TF-lite로 모델 사용후 시각화까지 한번 해보기 코드
* 위에 모델 바꿀때마다 바꿔줄꺼 바꿔보면서
 디버깅해봐야함

In [7]:
import cv2
import numpy as np
import tensorflow as tf
import math
import torch

In [8]:
############################
# 1) TFLite 모델 로드
############################
model_path = "/Users/parkjunhui/Desktop/Spresto/320_retinanet_Lens_tf/320_retinanet_Lens_float32.tflite"

interpreter = tf.lite.Interpreter(model_path=model_path)
interpreter.allocate_tensors()

input_details = interpreter.get_input_details()
output_details = interpreter.get_output_details()

############################
# 2) 이미지 로드 & 전처리
############################
image_path = "/Users/parkjunhui/Desktop/Spresto/test.jpg"
image_bgr = cv2.imread(image_path, cv2.IMREAD_COLOR)
if image_bgr is None:
    raise FileNotFoundError(f"이미지 로드 실패: {image_path}")

print("[DEBUG] Original image shape:", image_bgr.shape)

# Resize 640x640
resized_bgr = cv2.resize(image_bgr, (320,320))
# BGR -> RGB
# resized_rgb = cv2.cvtColor(resized_bgr, cv2.COLOR_BGR2RGB)
float_bgr = resized_bgr.astype(np.float32)
# Mean 빼기
print("[DEBUG] Before subtract mean => pixel sample:", float_bgr[0,0])  # 첫 픽셀 (B,G,R) 값
float_bgr[...,0] -= 103.53  # B
float_bgr[...,1] -= 116.28  # G
float_bgr[...,2] -= 123.675 # R
print("[DEBUG] After subtract mean => pixel sample:", float_bgr[0,0])
input_data = np.expand_dims(float_bgr, axis=0)
print("[DEBUG] input_data shape:", input_data.shape)


# TFLite 입력 세팅 & 추론
interpreter.set_tensor(input_details[0]['index'], input_data)
interpreter.invoke()

############################
# 3) TFLite 출력 얻기
############################
# RetinaNet: p3~p7, 총 10개 노드
# cls_logits: p3_cls_logits / p4_cls_logits / p5_cls_logits / p6_cls_logits / p7_cls_logits
# bbox_reg  : p3_bbox_reg  / p4_bbox_reg  / p5_bbox_reg  / p6_bbox_reg  / p7_bbox_reg

# 순서상 output_details[0]~[4]: cls_logits, [5]~[9]: bbox_reg
cls_logits_list = []
bbox_reg_list   = []
for i in range(3):
    cls_logits_list.append(interpreter.get_tensor(output_details[i]['index']))
    bbox_reg_list.append(interpreter.get_tensor(output_details[i+3]['index']))


print("=== [DEBUG] TFLite raw outputs ===")
for i, (cl, br) in enumerate(zip(cls_logits_list, bbox_reg_list)):
    print(f"P{i+3} cls: {cl.shape}, bbox: {br.shape}")


[DEBUG] Original image shape: (1920, 1080, 3)
[DEBUG] Before subtract mean => pixel sample: [36. 48. 58.]
[DEBUG] After subtract mean => pixel sample: [-67.53  -68.28  -65.675]
[DEBUG] input_data shape: (1, 320, 320, 3)
=== [DEBUG] TFLite raw outputs ===
P3 cls: (1, 80, 80, 9), bbox: (1, 80, 80, 36)
P4 cls: (1, 40, 40, 9), bbox: (1, 40, 40, 36)
P5 cls: (1, 20, 20, 9), bbox: (1, 20, 20, 36)


In [9]:
############################
# 4) 앵커 생성 함수 (예시)
############################
def generate_anchors_one_level(
    grid_height, grid_width, stride,
    base_size,
    aspect_ratios=[0.5, 1.0, 2.0],
    offset=0.0
):
    """
    scale 3개 사용: base_size*(2^(0/3)), base_size*(2^(1/3)), base_size*(2^(2/3))
    """
    # make 3 scales
    sizes = [base_size*(2**(i/3)) for i in range(3)]

    # 1) canonical anchors (9,4) around center (0,0)
    base_anchors = []
    for size in sizes:
        area = size*size
        for ar in aspect_ratios:
            w = math.sqrt(area/ar)
            h = ar * w
            x1, y1 = -w/2.0, -h/2.0
            x2, y2 = +w/2.0, +h/2.0
            base_anchors.append([x1,y1,x2,y2])
    base_anchors = np.array(base_anchors, dtype=np.float32)  # (9,4)

    # 2) shift grid
    shifts_x = (np.arange(grid_width) + offset)*stride
    shifts_y = (np.arange(grid_height) + offset)*stride
    shift_y, shift_x = np.meshgrid(shifts_y, shifts_x, indexing='ij')

    shift_x = shift_x.reshape(-1)
    shift_y = shift_y.reshape(-1)

    # shape=(HW,1,4)
    shifts = np.stack([shift_x, shift_y, shift_x, shift_y], axis=1)
    # broadcast sum
    K = shifts.shape[0]
    A = base_anchors.shape[0]
    all_anchors = (
        shifts.reshape(K,1,4) + base_anchors.reshape(1,A,4)
    ).reshape(K*A, 4)
    return all_anchors

In [10]:
def generate_anchors_p2to4(feature_shapes):
    """
    예: p3~p7에 대해,
    strides = [8,16,32,64,128]
    base_sizes = [32,64,128,256,512]
    feature_shapes: 예) [(80,80),(40,40),(20,20),(10,10),(5,5)]
    """
    strides = [4,8,16]
    base_sizes = [16,32,64]
    anchors_all = []
    for i, (H,W) in enumerate(feature_shapes):
        anchors_l = generate_anchors_one_level(
            grid_height=H, grid_width=W,
            stride=strides[i],
            base_size=base_sizes[i],
        )
        anchors_all.append(anchors_l)
    return anchors_all

In [11]:
############################
# 5) 디코딩 함수
############################
def decode_boxes(deltas, anchors):
    """
    deltas: shape=(N,4) -> (dx,dy,dw,dh)
    anchors: shape=(N,4) -> (x1,y1,x2,y2)
    output: shape=(N,4) -> decoded boxes
    """
    xa = 0.5*(anchors[:,0]+anchors[:,2])  # center x
    ya = 0.5*(anchors[:,1]+anchors[:,3])  # center y
    wa = anchors[:,2] - anchors[:,0]
    ha = anchors[:,3] - anchors[:,1]

    dx = deltas[:,0]
    dy = deltas[:,1]
    dw = deltas[:,2]
    dh = deltas[:,3]

    cx = xa + dx*wa
    cy = ya + dy*ha
    w = np.exp(dw)*wa
    h = np.exp(dh)*ha

    x1 = cx - 0.5*w
    y1 = cy - 0.5*h
    x2 = cx + 0.5*w
    y2 = cy + 0.5*h
    return np.stack([x1,y1,x2,y2], axis=1)

In [12]:
def iou_np(box1, boxes):
    """
    box1: shape=(4,),  (x1,y1,x2,y2)
    boxes: shape=(N,4)
    return: shape=(N,) 각 box와 box1의 IoU
    """
    # (x1,y1,x2,y2)에서 x2<x1인 케이스는 없다고 가정(정상 bbox)
    x1 = np.maximum(box1[0], boxes[:,0])
    y1 = np.maximum(box1[1], boxes[:,1])
    x2 = np.minimum(box1[2], boxes[:,2])
    y2 = np.minimum(box1[3], boxes[:,3])
    
    inter_w = np.maximum(0.0, x2 - x1)
    inter_h = np.maximum(0.0, y2 - y1)
    inter_area = inter_w * inter_h
    
    area1 = (box1[2] - box1[0]) * (box1[3] - box1[1])
    area2 = (boxes[:,2] - boxes[:,0]) * (boxes[:,3] - boxes[:,1])
    union = area1 + area2 - inter_area
    
    eps = 1e-6
    iou = inter_area / (union + eps)
    return iou

In [13]:
def nms_single_class(boxes, scores, iou_threshold=0.5):
    """
    boxes: shape=(N,4)
    scores: shape=(N,)
    각 box의 스코어를 내림차순으로 정렬 후, 
    IoU가 임계값을 넘는 box를 제거하는 "Greedy NMS" 구현.
    return: NMS 후 살아남은 박스의 인덱스(ascending이 아닌, 선택 순서).
    """
    # 스코어 내림차순 정렬
    order = np.argsort(-scores)  # descending
    keep_indices = []

    while len(order) > 0:
        i = order[0]
        keep_indices.append(i)
        
        if len(order) == 1:
            break

        # 현재 선택된 box i와의 IoU
        ious = iou_np(boxes[i], boxes[order[1:]])
        # IoU가 임계값 넘는 인덱스 제거
        mask = ious <= iou_threshold
        # order[1:][mask]만 남기기
        order = order[1:][mask]

    return keep_indices

In [14]:
def nms_classwise_np(boxes, scores, classes, iou_threshold=0.5, max_detections=100):
    """
    boxes:   shape=(N,4)
    scores:  shape=(N,)
    classes: shape=(N,)  (int)
    1) unique classes를 찾아, 각각에 대해 nms_single_class
    2) 최종 인덱스 모아, 스코어 기준으로 재정렬
    3) top max_detections
    return: 최종 살아남은 인덱스(오름차순 순서는 아님)
    """
    unique_classes = np.unique(classes)
    keep_indices_all = []

    for c in unique_classes:
        # 해당 클래스 c에 속하는 인덱스만 추출
        c_inds = np.where(classes == c)[0]
        if len(c_inds) == 0:
            continue
        c_boxes  = boxes[c_inds]
        c_scores = scores[c_inds]

        # 단일 클래스 NMS
        keep_c = nms_single_class(c_boxes, c_scores, iou_threshold)
        # 실제 전역 인덱스로 변환
        keep_indices_all.extend(c_inds[k] for k in keep_c)

    if len(keep_indices_all) == 0:
        return np.array([], dtype=np.int64)

    # keep_indices_all를 스코어 기준 내림차순 정렬
    #  -> top max_detections
    final_scores = scores[keep_indices_all]
    desc_order = np.argsort(-final_scores)
    desc_order = desc_order[:max_detections]  # 상위 max_detections
    final_keep = np.array(keep_indices_all)[desc_order]

    return final_keep

In [15]:
import numpy as np
import math

def retinanet_postprocess_tflite(
    cls_logits_list,  # list of np.array: p3->p7, shape=(1,H,W,720)
    bbox_reg_list,    # list of np.array: p3->p7, shape=(1,H,W,36)
    conf_thresh=0.05,
    # iou_thresh=0.5,
    nms_thresh=0.5,  
    topk_candidates=1000,
    max_detections=100
):
    feature_shapes = [cl.shape[1:3] for cl in cls_logits_list]  # [(80,80),(40,40),(20,20),(10,10),(5,5)]
    anchors_list = generate_anchors_p2to4(feature_shapes)  # (사용자 정의 함수)

    all_boxes   = []
    all_scores  = []
    all_classes = []

    num_classes = 1
    anchors_per_loc = 9

    for level_i in range(len(cls_logits_list)):
        cls_logits = cls_logits_list[level_i]  # (1,H,W,720)
        bbox_regs  = bbox_reg_list[level_i]    # (1,H,W,36)
        anchors_l  = anchors_list[level_i]     # (H*W*9,4)

        H,W = cls_logits.shape[1], cls_logits.shape[2]
        cls_logits_2d = cls_logits[0].reshape(H*W, 9)
        bbox_regs_2d  = bbox_regs[0].reshape(H*W, 36)

        # 시그모이드
        cls_conf_2d = 1.0/(1.0+np.exp(-cls_logits_2d))

        for a_i in range(anchors_per_loc):
            cls_part = cls_conf_2d[:, a_i*num_classes : (a_i+1)*num_classes]  # (HW,80)
            bbox_part= bbox_regs_2d[:, a_i*4 : (a_i+1)*4]                    # (HW,4)

            # flatten => (HW*80)
            flat_scores = cls_part.reshape(-1)
            # threshold
            inds = np.where(flat_scores > conf_thresh)[0]
            
            if len(inds)==0:
                continue

            # loc_ids / class_ids를 먼저 계산
            class_ids = inds % num_classes
            loc_ids   = inds // num_classes
            selected_scores = flat_scores[inds]

            # 아직 수량이 너무 많다면, 예시로 첫 5개만 찍는다
            # print(f"[DEBUG] level={level_i}, anchor_idx={a_i}, #selected={len(inds)}")
            
            # deltas = bbox_part[loc_ids], anchors_l[loc_ids]
            # 첫 몇개만
            print("   deltas[:5] =\n", bbox_part[loc_ids][:5])
            print("   anchors[:5] =\n", anchors_l[loc_ids][:5])


            anchor_sel = anchors_l[loc_ids]  # (N,4)
            delta_sel  = bbox_part[loc_ids]  # (N,4)

            decoded = decode_boxes(delta_sel, anchor_sel)  # (N,4)

            print("   decoded[:5] =\n", decoded[:5])
            # 만약 decoded[:5] 중 y1 < 0이 있는지 확인
            y1s = decoded[:5,1]
            if np.any(y1s < 0):
                print("   [WARNING] Some y1 are negative =>", y1s)
                

            all_boxes.append(decoded)
            all_scores.append(selected_scores)
            all_classes.append(class_ids)

    if len(all_boxes)==0:
        return np.zeros((0,4)), np.array([]), np.array([])

    all_boxes   = np.concatenate(all_boxes, axis=0)    # shape=(M,4)
    all_scores  = np.concatenate(all_scores, axis=0)   # shape=(M,)
    all_classes = np.concatenate(all_classes, axis=0)  # shape=(M,)

    # topk
    if all_scores.size > topk_candidates:
        idx_top = np.argpartition(all_scores, -topk_candidates)[-topk_candidates:]
        all_boxes   = all_boxes[idx_top]
        all_scores  = all_scores[idx_top]
        all_classes = all_classes[idx_top]

    # === (바뀐 부분) 클래스별 NMS를 직접 수행 ===
    keep = nms_classwise_np(all_boxes, all_scores, all_classes,
                            iou_threshold=nms_thresh,
                            max_detections=max_detections)

    final_boxes   = all_boxes[keep]
    final_scores  = all_scores[keep]
    final_classes = all_classes[keep]

    return final_boxes, final_scores, final_classes

In [15]:

############################
# 7) 후처리 실행
############################
final_boxes, final_scores, final_classes = retinanet_postprocess_tflite(
    cls_logits_list, bbox_reg_list,
    conf_thresh=0.05,
    nms_thresh=0.3,
    topk_candidates=1000,
    max_detections=3
)

print("=== Detection Results ===")
for i in range(len(final_boxes)):
    box = final_boxes[i]    # (x1,y1,x2,y2)
    score = final_scores[i]
    clsid = final_classes[i]
    print(f"{i}: class={clsid}, score={score:.3f}, box={box}")

   deltas[:5] =
 [[ 0.02438388  0.00164803  0.15431586  0.11322363]
 [-0.13588993  0.03981597  0.14943227  0.09400295]]
   anchors[:5] =
 [[276.68629169  18.34314585 299.31370831  29.65685415]
 [280.          16.         296.          32.        ]]
   decoded[:5] =
 [[275.35023738  17.68363428 301.75325088  30.35365647]
 [276.53636289  15.84855193 295.11515927  33.42555922]]
   deltas[:5] =
 [[ 0.03122225 -0.01056861  0.3687736  -0.19439352]]
   anchors[:5] =
 [[276.68629169  18.34314585 299.31370831  29.65685415]]
   decoded[:5] =
 [[272.34730743  19.22295028 305.06565047  28.53790941]]
   deltas[:5] =
 [[ 0.04116405 -0.02305134  0.11439863  0.03839124]
 [-0.1165406   0.01073337  0.15578783  0.02411323]]
   anchors[:5] =
 [[276.68629169  18.34314585 299.31370831  29.65685415]
 [280.          16.         296.          32.        ]]
   decoded[:5] =
 [[276.24651749  17.86095333 301.61635464  29.61745439]
 [276.78672421  15.97648337 295.48397648  32.3669844 ]]
   deltas[:5] =
 [[-0.00231

In [16]:
def clip_boxes_to_image(boxes, img_width, img_height):
    """
    boxes: np.array of shape (N,4) [x1,y1,x2,y2]
    """
    boxes[:,0] = np.maximum(boxes[:,0], 0.0)            # x1 >= 0
    boxes[:,1] = np.maximum(boxes[:,1], 0.0)            # y1 >= 0
    boxes[:,2] = np.minimum(boxes[:,2], img_width)      # x2 <= width
    boxes[:,3] = np.minimum(boxes[:,3], img_height)     # y2 <= height
    return boxes

In [21]:
import cv2
import numpy as np

def visualize_detections(
    image_bgr, 
    final_boxes,     # shape=(N,4)
    final_scores,    # shape=(N,)
    final_classes,   # shape=(N,)
    class_names=None # 예: ["person", "car", ...] 
):
    """
    image_bgr: 원본 또는 리사이즈된 BGR 이미지 (H,W,3)
    final_boxes: (x1,y1,x2,y2), float
    final_scores: confidence
    final_classes: 정수 클래스 ID
    class_names: 클래스 이름 리스트 (옵션)
    """
    img_h, img_w = image_bgr.shape[:2]
    
    # 1) 박스를 이미지 범위에 클리핑
    boxes_clipped = clip_boxes_to_image(final_boxes.copy(), img_w, img_h)

    # 2) 박스 그려넣기
    for i in range(len(boxes_clipped)):
        box = boxes_clipped[i]
        score = final_scores[i]
        clsid = final_classes[i]
        
        x1, y1, x2, y2 = box.astype(int)  # 정수 좌표 변환
        print("Draw box =>", x1, y1, x2, y2)
        
        # 선택적으로, 너무 작은 박스는 건너뛸 수도 있음
        if (x2 - x1) < 1 or (y2 - y1) < 1:
            continue

        # 색상(임시): 클래스 ID에 따라 임의 색 지정
        color = (0, 255, 255)  # 노랑(BGR) 등
        # 혹은 clsid를 이용해 여러 색 중 선택할 수도 있음
        
        # 사각형 그리기
        cv2.rectangle(
            image_bgr,
            (x1, y1),
            (x2, y2),
            color,
            thickness=2
        )
        
        # 라벨 텍스트
        if class_names is not None and 0 <= clsid < len(class_names):
            cls_name = class_names[clsid]
        else:
            cls_name = f"cls:{clsid}"
        
        label_text = f"{cls_name} {score:.2f}"
        print(label_text)
        
        # 텍스트 표시 위치 (x1,y1-5) 등
        cv2.putText(
            image_bgr,
            label_text,
            (x2, max(y2 - 5, 0)),
            cv2.FONT_HERSHEY_SIMPLEX,
            0.6,      # 폰트 스케일
            color,
            2         # 두께
        )
    
    return image_bgr

In [22]:
draw_img = resized_bgr.copy()  # 원본 유지
draw_img = visualize_detections(
    draw_img,
    final_boxes, final_scores, final_classes, class_names = ["Hidden_lens", "Machine_Mini", "Machine_Plug"] 
    
# index=2일 때 "Car"
)

print(final_classes)

cv2.imwrite("detection_result_1.jpg", draw_img)

NameError: name 'final_boxes' is not defined